In [1]:
import pandas as pd
import numpy as np

In [60]:
# Density data from 2010 census
# https://blog.splitwise.com/2014/01/06/free-us-population-density-and-unemployment-rate-by-zip-code/
density = pd.read_csv('data/zip_pop-density_2010.csv')
density.rename(columns={'Zip/ZCTA': 'zip', '2010 Population': '2010pop', 'Land-Sq-Mi': 'land_sq_mi', 'Density Per Sq Mile': 'zip_density'}, inplace=True)
density.zip = density.zip.astype('str').str.zfill(5)
#density.set_index('zip', inplace=True)
density.head()

,zip,2010pop,land_sq_mi,zip_density
0,00601,0,64.348,0.0
1,00602,0,30.613,0.0
2,00603,0,31.616,0.0
3,00606,0,42.309,0.0
4,00610,0,35.916,0.0


In [61]:
# Income data from IRS
# https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2015-zip-code-data-soi
income = pd.read_csv('data/zip_IRS_income_2015.csv')
income.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,836320.0,481570.0,109790.0,233260.0,455560.0,1356760.0,...,373410.0,328469.0,0.0,0.0,0.0,0.0,61920.0,48150.0,732670.0,1933120.0
1,1,AL,0,2,494830.0,206630.0,146250.0,129390.0,275920.0,1010990.0,...,395880.0,965011.0,0.0,0.0,0.0,0.0,73720.0,107304.0,415410.0,1187403.0
2,1,AL,0,3,261250.0,80720.0,139280.0,36130.0,155100.0,583910.0,...,251490.0,1333418.0,0.0,0.0,0.0,0.0,64200.0,139598.0,193030.0,536699.0
3,1,AL,0,4,166690.0,28510.0,124650.0,10630.0,99950.0,423990.0,...,165320.0,1414283.0,0.0,0.0,0.0,0.0,45460.0,128823.0,116440.0,377177.0
4,1,AL,0,5,212660.0,19520.0,184320.0,4830.0,126860.0,589490.0,...,212000.0,3820152.0,420.0,168.0,60.0,31.0,83330.0,421004.0,121570.0,483682.0


In [62]:
# format income data
'''
agi_stub: 
1 = $1 under $25,000
2 = $25,000 under $50,000
3 = $50,000 under $75,000
4 = $75,000 under $100,000
5 = $100,000 under $200,000
6 = $200,000 or more

N1: Number of returns
ELDERLY: Number of elderly returns. Includes primary taxpayers 60 years of age and older.
A00100: Adjust gross income (AGI). Does not include returns with adjusted gross deficit.
N02650: Number of returns with total income
A02650: Total income amount
N00200: Number of returns with salaries and wages
A00200: Salaries and wages amount
'''
keep_cols = ['zipcode', 'agi_stub', 'N1', 'ELDERLY', 'A00100', 'N02650', 'A02650',  'N00200', 'A00200']
income_no_agi = income[keep_cols].groupby('zipcode').sum()
income_no_agi['total_income_per_return'] = income_no_agi['N02650']/income_no_agi['A02650']
income_no_agi['total_wages_per_return'] = income_no_agi['N00200']/income_no_agi['A00200']

#rename columns
income_no_agi.reset_index(inplace=True)
income_no_agi.rename(columns={'zipcode': 'zip', 'N1': 'n_returns', 'ELDERLY': 'eld_returns', 'A00100': 'agi', 'N02650': 'n_return_total_inc', 'A02650': 'total_income_amt', 'N00200': 'n_returns_wages', 'A00200': 'wages_amt'}, inplace=True)

income_no_agi.zip = income_no_agi.zip.astype('str').str.zfill(5) # convert zipcodes to 5-digit string

income_no_agi.head()

,zip,agi_stub,n_returns,eld_returns,agi,n_return_total_inc,total_income_amt,n_returns_wages,wages_amt,total_income_per_return,total_wages_per_return
0,00000,1071,146918270.0,33798140.0,1.020119e+10,146918030.0,1.034457e+10,122752180.0,6.982007e+09,0.014202,0.017581
1,01001,21,8950.0,2610.0,4.987380e+05,8950.0,5.047740e+05,7380.0,3.804340e+05,0.017731,0.019399
2,01002,21,9670.0,3030.0,7.783760e+05,9670.0,7.964750e+05,7790.0,4.584000e+05,0.012141,0.016994
3,01005,21,2300.0,610.0,1.367440e+05,2300.0,1.383090e+05,1980.0,1.054320e+05,0.016629,0.018780
4,01007,21,7490.0,1940.0,5.297960e+05,7490.0,5.375090e+05,6440.0,4.091800e+05,0.013935,0.015739


In [63]:
# Merge income and density data for each zipcode
zip_data = pd.merge(density, income_no_agi, on='zip')
zip_data.head()

,zip,2010pop,land_sq_mi,zip_density,agi_stub,n_returns,eld_returns,agi,n_return_total_inc,total_income_amt,n_returns_wages,wages_amt,total_income_per_return,total_wages_per_return
0,01001,16769,11.442,1465.565461,21,8950.0,2610.0,498738.0,8950.0,504774.0,7380.0,380434.0,0.017731,0.019399
1,01002,29049,55.043,527.751031,21,9670.0,3030.0,778376.0,9670.0,796475.0,7790.0,458400.0,0.012141,0.016994
2,01005,5079,44.242,114.800416,21,2300.0,610.0,136744.0,2300.0,138309.0,1980.0,105432.0,0.016629,0.018780
3,01007,14649,52.643,278.270615,21,7490.0,1940.0,529796.0,7490.0,537509.0,6440.0,409180.0,0.013935,0.015739
4,01008,1263,53.796,23.477582,21,620.0,190.0,39757.0,620.0,40213.0,520.0,28845.0,0.015418,0.018027


In [64]:
# export data
zip_data.to_csv('data/zip_data.csv', sep='\t')